In [5]:
#input: telegram channel called @Funda_news (for now its just 200 last news from this channel in StreamData.csv
# ke yeki yeki bad az 30 sanie stream mishan.)
#output: 2-Columns to preprocessing part.
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from kafka import KafkaProducer,KafkaConsumer
import json, datetime , csv, time
from pyspark.ml.feature import VectorAssembler
import pandas as pd
from tqdm import tqdm

pathStreamData = r'C:\Users\AhmaDGoly\Desktop\Arshad\TERM 2\Big Data\Final\Codes\StreamData.csv' #moheme.
TimeToSendEachRow = 30 #seconds
kafkaPort = 19092
crawlerTopic = 'crawler'


In [3]:
def current_time():
    current_datetime = datetime.datetime.now()
    return current_datetime.strftime("[%Y-%m-%d_%H:%M:%S]")
    
producer = KafkaProducer(bootstrap_servers=[f'127.0.0.1:{kafkaPort}'])
df = pd.read_csv(pathStreamData, encoding='utf-8-sig')

In [6]:
i = 1
print(f"{current_time()}: Started Sending 2-column rows to kafka on topic = {crawlerTopic}.")
for index, row in df.iterrows():
    print(f"{current_time()}: Streamed {i} row(s) on topic = {crawlerTopic}.", end='\r')
    if i == len(df): print(f"{current_time()}: Streamed {i} row(s) on topic = {crawlerTopic}.")
    json_row = json.dumps(row.to_dict())
    producer.send(crawlerTopic, value=json_row.encode('utf-8'))
    i = i + 1
    time.sleep(TimeToSendEachRow)
print(f"{current_time()}: Streaming data complete.")


[2023-07-13_18:58:47]: Started Sending 2-column rows to kafka on topic = crawler.
[2023-07-13_18:58:47]: Streamed 200 row(s) on topic = crawler.
[2023-07-13_18:58:47]: Streaming data complete.


In [18]:
producer.close()

In [ ]:
#Original Code: getting the last news from channel using Telegram API. it only works on google colab because I don't have a good vpn.
#credit: Amir yousefi (https://github.com/amiryousefi/telegram-analysis/tree/master)
'''
import configparser
import json
import asyncio
from datetime import date, datetime

from telethon import TelegramClient
from telethon.errors import SessionPasswordNeededError
from telethon.tl.functions.messages import (GetHistoryRequest)
from telethon.tl.types import (
    PeerChannel
)


# some functions to parse json date
class DateTimeEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, datetime):
            return o.isoformat()

        if isinstance(o, bytes):
            return list(o)

        return json.JSONEncoder.default(self, o)


# Setting configuration values
api_id = ...
api_hash = ...

api_hash = str(api_hash)

phone = +989910802462
username = mrfreak77

# Create the client and connect
client = TelegramClient(username, api_id, api_hash)

async def main(phone):
    await client.start()
    print("Client Created")
    # Ensure you're authorized
    if await client.is_user_authorized() == False:
        await client.send_code_request(phone)
        try:
            await client.sign_in(phone, input('Enter the code: '))
        except SessionPasswordNeededError:
            await client.sign_in(password=input('Password: '))

    me = await client.get_me()

    user_input_channel = input('enter entity(telegram URL or entity id):')

    if user_input_channel.isdigit():
        entity = PeerChannel(int(user_input_channel))
    else:
        entity = user_input_channel

    my_channel = await client.get_entity(entity)

    offset_id = 0
    limit = 100
    all_messages = []
    total_messages = 0
    total_count_limit = 0

    while True:
        print("Current Offset ID is:", offset_id, "; Total Messages:", total_messages)
        history = await client(GetHistoryRequest(
            peer=my_channel,
            offset_id=offset_id,
            offset_date=None,
            add_offset=0,
            limit=limit,
            max_id=0,
            min_id=0,
            hash=0
        ))
        if not history.messages:
            break
        messages = history.messages
        for message in messages:
            all_messages.append(message.to_dict())
        offset_id = messages[len(messages) - 1].id
        total_messages = len(all_messages)
        if total_count_limit != 0 and total_messages >= total_count_limit:
            break

    with open('channel_messages.json', 'w') as outfile:
        json.dump(all_messages, outfile, cls=DateTimeEncoder)

with client:
    client.loop.run_until_complete(main(phone))

'''
